# Линейное программирование

In [1]:
# https://ocw.mit.edu/courses/sloan-school-of-management/15-071-the-analytics-edge-spring-2017/linear-optimization/

# https://ocw.mit.edu/courses/sloan-school-of-management/15-053-optimization-methods-in-management-science-spring-2013/lecture-notes/

In [2]:
import numpy as np
import cvxpy
np.set_printoptions(suppress=True)

In [4]:
# https://ru.wikipedia.org/wiki/Задача_о_рюкзаке

values = np.array([4, 2, 1, 7, 3, 6])
weights = np.array([5, 9, 8, 2, 6, 5])
C = 15
n = 6

x = cvxpy.Variable(shape=n, boolean=True)
constraints = [
               weights @ x <= C,
              #  x >= 0
               ]
total_value = values @ x
problem = cvxpy.Problem(cvxpy.Maximize(total_value), constraints=constraints)
print(problem.solve())
print(x.value)

17.0
[1. 0. 0. 1. 0. 1.]


In [2]:
# Задача № 2 - blending
c = np.array([
              [0.15, 0.15, 0.17],
              [0.10, 0.08, 0.09],
              [0.007, 0.003, 0.005],
              [300, 200, 250],
              ])

x = cvxpy.Variable(shape=(3, 1), integer=False)

# Ограничения
constraint = [
              c[0] @ x >= 0.16,
              c[0] @ x <= 0.18, 
              c[1] @ x <= 0.09, 
              c[2] @ x <= 0.005, 
              x >= 0,
              cvxpy.sum(x[:, :]) == 1
              ]

# минимизируемая функция        
total_cost = c[3] @ x
problem = cvxpy.Problem(cvxpy.Minimize(total_cost), constraints=constraint)
print('Мин. стоимость 1 единицы состава ($):')
print(np.round(problem.solve(solver='ECOS_BB'), 2))
print('Содержание каждого из материалов в 1 единице смеси (%)')
print(np.round(100*x.value, 2))

Мин. стоимость 1 единицы состава ($):
225.0
Содержание каждого из материалов в 1 единице смеси (%)
[[-0.]
 [50.]
 [50.]]


In [1]:
# Входные данные: a_i -- массив (3 x 1), пропорции, в которых нужно брать составы.

# Конкретный пример пропорций из прошлой задачи (в процентах): 0, 50, 50.

# Ограничения (в тоннах) по объему каждого из составов: 150, 100, 75.

# Ограничение на загрузку одной печи -- 200 тонн.

# Целевая функция: у = a_1 x + a_2 x + a_1 x, где x - сколько раз нужно брать a_i-тое 
# частей каждого состава.

# Например, с данными из примера: нужно взять 0 частей состава 1, 50 частей состава 2 и 50 
# частей состава 3 по 1.5 раза каждого.

# Для этого примера я ожидаю ответ: максимум - 150, потонный состав -- 0, 75 и 75 тонн.

# Если поменять пропорции на, к примеру, 0, 80 и 20, то я ожидаю такой рассчитанный 
# вручную результат: 125 тонн макс. загрузка, по 0, 100 и 25 тоннам соответствующих составов.

In [4]:
# # Задача № 3
# Здесь нужно идти через максимум получаемой прибыли. 
# По-хорошему необходимо сначала решить целиком вторую, найти в ней самый оптимальный 
# (= дешевый состав). Затем попытаться для начала применить его в решении 3-ей задачи.
# Однако если печь не загружена полностью, нужно перерешать задачу линейным программированием, 
# ведь тогда мы может быть сможем ещё увеличить прибыль, пусть используя и не 
# самый дешевый (= найденный в задаче 2) состав.
import numpy as np
import cvxpy
np.set_printoptions(suppress=True)

# a = np.array([0, 50, 50])
a = np.array([0, 80, 20])

x = cvxpy.Variable(shape=(3,), integer=False)

# Constraints
constraint = [
              x >= 0,
              x[0] == x[1],
              x[1] == x[2],
              cvxpy.multiply(a[0], x[0]) <= 150,
              cvxpy.multiply(a[1], x[1]) <= 100,
              cvxpy.multiply(a[2], x[2]) <= 75,
              a @ x <= 200,
              ]

# Objective function        
total_load = a @ x
problem = cvxpy.Problem(cvxpy.Maximize(total_load), constraints=constraint)
print('Макс. загрузка печи (тонн):')
print(np.round(problem.solve(solver='ECOS_BB'), 2))
print('Сколько тонн каждого состава нужно взять (тонны):')
print(np.multiply(x.value, a))

Макс. загрузка печи (тонн):
125.0
Сколько тонн каждого состава нужно взять (тонны):
[  0. 100.  25.]


In [1]:
# Задача № 3 (с х из одного элемента вместо вектора из трех элементов)
import numpy as np
import cvxpy
np.set_printoptions(suppress=True)

# a = np.array([0, 1, 1])
# a = np.array([0, 0.5, 0.5])
a = np.array([0, 50, 50])
# a = np.array([0, 80, 20])
# a = np.array([0, 4, 1])

x = cvxpy.Variable(shape=(1,), integer=False)

# Constraints
constraint = [
              x >= 0,
              cvxpy.multiply(a[0], x) <= 150,
              cvxpy.multiply(a[1], x) <= 100,
              cvxpy.multiply(a[2], x) <= 75,
              cvxpy.sum(cvxpy.multiply(a, x)) <= 200,
              ]

# Objective function        
total_load = cvxpy.sum(cvxpy.multiply(a, x))
problem = cvxpy.Problem(cvxpy.Maximize(total_load), constraints=constraint)
print('Макс. загрузка печи (тонн):')
print(np.round(problem.solve(solver='ECOS_BB'), 2))
print('Сколько тонн каждого состава нужно взять (тонны):')
print(np.round(np.multiply(x.value, a), 2))
# Макс. загрузка печи (тонн):
# 150.0
# Сколько тонн каждого состава нужно взять (тонны):
# [ 0. 75. 75.]

# Макс. загрузка печи (тонн):
# 125.0
# Сколько тонн каждого состава нужно взять (тонны):
# [  0. 100.  25.]

Макс. загрузка печи (тонн):
150.0
Сколько тонн каждого состава нужно взять (тонны):
[ 0. 75. 75.]


In [2]:
# Задача 4 (часть 1)
import numpy as np
import cvxpy
np.set_printoptions(suppress=True)

a = np.array([[0, 3, 9, 11],   # кол-во единиц i-го ресурса, 
              [3, 5, 7, 0],    # идущего на производство
              [4, 8, 0, 13],   # единицы товара по j-ой технологии
              [15, 20, 17, 21] # прибыль от ед. товара по j-ой технологии
              ])

# x_j -- объем выпуска товара по j-ой технологии
x = cvxpy.Variable(shape=(4,), integer=True)

constraint = [
    x >= 0,
    a[0] @ x <= 300,  # максимальный запас 1-го ресурса
    a[1] @ x <= 400,  # максимальный запас 2-го ресурса
    a[2] @ x <= 450,  # максимальный запас 3-го ресурса
    ]

# Objective function        
total_profit = a[3] @ x
problem = cvxpy.Problem(cvxpy.Maximize(total_profit), constraints=constraint)
print('Макс. прибыль ($):')
print(np.round(problem.solve(solver='ECOS_BB'), 2))
print('Сколько единиц товара по каждой технологии предлагается производить:')
print(np.round(x.value))

Макс. прибыль ($):
1833.0
Сколько единиц товара по каждой технологии предлагается производить:
[112.   0.   9.   0.]


In [12]:
# Задача 4 (часть 2) Переборная часть во второй части - неверное решение!
import numpy as np
import cvxpy
np.set_printoptions(suppress=True)


def production_plan(a_matrix, b_vector):
    """Получаем расход ресурсов, прибыль, лимит запасов, возвращаем прибыль и план."""
     # x_j - объем выпуска товара по j-ой технологии
    x = cvxpy.Variable(shape=(4,), integer=True)

    constraint = [
        a_matrix[0] @ x <= b_vector[0],  # максимальный запас 1-го ресурса
        a_matrix[1] @ x <= b_vector[1],  # максимальный запас 2-го ресурса
        a_matrix[2] @ x <= b_vector[2],  # максимальный запас 3-го ресурса
        x >= 0,
        ]

    # Objective function        
    ttl_profit = a_matrix[3] @ x
    problem = cvxpy.Problem(cvxpy.Maximize(ttl_profit), constraints=constraint)

    max_profit = np.round(problem.solve(solver='ECOS_BB'), 2)
    prod_plan = np.round(x.value)

    return max_profit, prod_plan

In [13]:
# Входные данные
a = np.array([[0, 3, 9, 11],   # кол-во единиц i-го ресурса, 
              [3, 5, 7, 0],    # идущего на производство
              [4, 8, 0, 13],   # единицы товара по j-ой технологии
              [15, 20, 17, 21] # прибыль от ед. товара по j-ой технологии
              ])

# b_i -- максимальный запас i-го ресурса
b = np.array([[300, 400, 450],  # базовый сценарий
              [302, 400, 450],  # докупаем 2 единицы 1-го вида ресурса
              [300, 402, 450],  # докупаем 2 единицы 2-го вида ресурса
              [300, 400, 452],  # докупаем 2 единицы 3-го вида ресурса
              ])
# сначала выполняем предыдующий план, затем смотрим, что будет если докупить чего-нибудь
# b = np.array([
#               # [300, 400, 450],  # базовый сценарий
#               [221, 1, 2],  # докупаем 2 единицы 1-го вида ресурса
#               [219, 3, 2],  # докупаем 2 единицы 2-го вида ресурса
#               [219, 1, 4],  # докупаем 2 единицы 3-го вида ресурса
#               ])

# доп. затраты по каждому сценарию
extra_cost = [0.36, 0.44, 0.38]

In [14]:
# проверить, что я ничего не сломал и по базовому сценарию такой же результат, что и в части 1
res = production_plan(a, b[0])
print('Макс. прибыль ($):')
print(res[0])
print('Сколько единиц товара по каждой технологии предлагается производить:')
print(res[1])

Макс. прибыль ($):
1833.0
Сколько единиц товара по каждой технологии предлагается производить:
[112.   0.   9.   0.]


In [15]:
# Описание всех сценариев
count = 0
scenario_table = []
for scenario_i, extra_cost_item in zip(b, extra_cost):
    print(f'Сценарий {count}. Запасы: {scenario_i} Допзатраты: {extra_cost_item}')
    profit, plan = production_plan(a, scenario_i)
    net_profit = profit - extra_cost_item
    scenario_table.append([net_profit, plan])
    print(f'Чистая прибыль: {net_profit} План: {plan}', end='\n\n')
    count += 1

Сценарий 0. Запасы: [300 400 450] Допзатраты: 0.36
Чистая прибыль: 1832.64 План: [112.   0.   9.   0.]

Сценарий 1. Запасы: [302 400 450] Допзатраты: 0.44
Чистая прибыль: 1832.56 План: [112.   0.   9.   0.]

Сценарий 2. Запасы: [300 402 450] Допзатраты: 0.38
Чистая прибыль: 1835.62 План: [106.  -0.  12.   2.]



In [11]:
# Выбираем лучший сценарий
scenario_table = np.array(scenario_table, dtype='object')
idx_best_scenario = scenario_table[:,0].argmax()
best_profit, best_plan = scenario_table[idx_best_scenario]
print(f'Номер лучшего сценария: {idx_best_scenario}')
print(f'Лучшая прибыль: {best_profit}, Лучший план: {best_plan}')

Номер лучшего сценария: 2
Лучшая прибыль: 1843.76, Лучший план: [112.   0.   9.   0.]


In [1]:
# Задача 5
import numpy as np
import cvxpy
np.set_printoptions(suppress=True)


def distribute(a, weights: list):
    """Получаем матрицу прибыли и веса, а возвращем макс. прибыль и распределение."""
    resources_num = len(weights)  # количество ресурсов
    x = cvxpy.Variable(shape=(a.shape[0], a.shape[1]), integer=True)

    constraints = [
        cvxpy.sum(x, axis=0) @ weights <= resources_num,
        cvxpy.sum(x, axis=1) <= 1,  # 1 вариант выбора распределения для каждого агрегата
        x <= 1,
        x >= 0
        ]
    # Objective function        
    ttl_profit = cvxpy.sum(cvxpy.multiply(a, x))  # обычное умножение, не матричное
    problem = cvxpy.Problem(cvxpy.Maximize(ttl_profit), constraints=constraints)

    max_profit = np.round(problem.solve(solver='ECOS_BB'), 1)
    distribution = np.round(x.value)

    return max_profit, distribution

In [96]:
# часть 1
a = np.array([[5, 7.5, 9, 11],
              [4, 5.5, 8, 10],
              [6, 8, 10, 12],
              [4.5, 7, 9, 11]
              ])
print('a =\n', a)
weights = [1, 2, 3, 4]
res = distribute(a, weights)
print(f'\nMax profit = $ {res[0]}')
print(f'Distribution:\n {res[1]}')

a =
 [[ 5.   7.5  9.  11. ]
 [ 4.   5.5  8.  10. ]
 [ 6.   8.  10.  12. ]
 [ 4.5  7.   9.  11. ]]

Max profit = $ 19.5
Distribution:
 [[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]


In [97]:
# часть 2
# убираем 1 единицу ресурса, то есть четвертую колонку
a = np.array([[5, 7.5, 9, 11],
              [4, 5.5, 8, 10],
              [6, 8, 10, 12],
              [4.5, 7, 9, 11]
              ])
a = a[:, 0:3]
print('a =\n', a)
weights = [1, 2, 3]
res = distribute(a, weights)
print(f'\nMax profit = $ {res[0]}')
print(f'Distribution:\n {res[1]}')

a =
 [[ 5.   7.5  9. ]
 [ 4.   5.5  8. ]
 [ 6.   8.  10. ]
 [ 4.5  7.   9. ]]

Max profit = $ 15.5
Distribution:
 [[ 1.  0. -0.]
 [ 0. -0. -0.]
 [ 1.  0. -0.]
 [ 1. -0. -0.]]


In [98]:
# часть 3 - пример 1
# убираем 1-ый агрегат, например, первый, а кол-во ресурсов исходное, т.е. 4
a = np.array([[5, 7.5, 9, 11],
              [4, 5.5, 8, 10],
              [6, 8, 10, 12],
              [4.5, 7, 9, 11]
              ])
a = a[1:, :]
print('a = \n', a)
weights = [1, 2, 3, 4]
res = distribute(a, weights)
print(f'\nMax profit = $ {res[0]}')
print(f'Distribution:\n {res[1]}')

a = 
 [[ 4.   5.5  8.  10. ]
 [ 6.   8.  10.  12. ]
 [ 4.5  7.   9.  11. ]]

Max profit = $ 17.0
Distribution:
 [[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]]


In [99]:
# часть 3 - пример 2
# убираем 2-ой агрегат; кол-во ресурсов исходное, т.е. 4
a = np.array([[5, 7.5, 9, 11],
              [4, 5.5, 8, 10],
              [6, 8, 10, 12],
              [4.5, 7, 9, 11]
              ])
a = np.vstack((a[0:1, :], a[2:, :]))
print('a =\n',  a)
weights = [1, 2, 3, 4]
res = distribute(a, weights)
print(f'\nMax profit = $ {res[0]}')
print(f'Distribution:\n {res[1]}')

a =
 [[ 5.   7.5  9.  11. ]
 [ 6.   8.  10.  12. ]
 [ 4.5  7.   9.  11. ]]

Max profit = $ 18.0
Distribution:
 [[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]]


In [2]:
a = np.array([[5, 7.5, 9, 11],
              [4, 5.5, 8, 10],
              [6, 8, 10, 12],
              # [4.5, 7, 9, 11]
              ])
print('a =\n', a)
weights = [1, 2, 3, 4]
res = distribute(a, weights)
print(f'\nMax profit = $ {res[0]}')
print(f'Distribution:\n {res[1]}')

a =
 [[ 5.   7.5  9.  11. ]
 [ 4.   5.5  8.  10. ]
 [ 6.   8.  10.  12. ]]

Max profit = $ 17.5
Distribution:
 [[0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]


# Динамическое програмирование

In [ ]:
from pprint import pprint
import numpy as np
import pandas as pd

In [ ]:
# задача № 5 - правильное решение с помощью динамического программирования
def distribute_two(a):
    """Return max profit list and distribution list for 2 banks."""
    max_list = []
    opt_list = []

    for i in range(len(a[0])):
        max = a[0][i] + a[1][0]
        opt = [i, 0]
        for j in range(i+1):
            test_max = a[0][i-j] + a[1][j]
            if test_max > max:
                max = test_max
                opt = [i-j, j]
        max_list.append(max)
        opt_list.append(opt)
    return max_list, opt_list


# a = [[0, 5, 7.5, 9, 10],
#      [0, 4, 5.5, 8, 10]
#      ]
# b = [[0, 5, 9, 11.5, 13],
#      [0, 6, 8, 10, 12]
#      ]

# distribute_two(a)
# distribute_two(b)

In [ ]:
def distribute(a):
    """Return max profits and distribution of investments.
    
    a : a profit matrix A x B, rows are banks, columns are investments
        a_ij - profit j from bank i 
    """
    a = np.hstack((np.zeros((len(a), 1)), a))
    dist_table = list(a[:2, :])

    F_12_max, F_12_distr = distribute_two(a[0:2, :])
    dist_table.append(F_12_max)
    dist_table.append(F_12_distr)
    for row in a[2:, :]:
        # print(row)
        dist_table.append(row)
        F_123 = distribute_two([dist_table[-3], row])
        F_123_dist = []
        for i in F_123[1]:
            F_123_dist.append(dist_table[-2][i[0]] + [i[1]])
        dist_table.append(F_123[0])    # add maximum profits
        dist_table.append(F_123_dist)  # add optimal distribution

    return dist_table

# a = [[5, 7.5, 9, 10],
#   [4, 5.5, 8, 10],
#   [6, 8, 10, 12],
#   [4.5, 7, 9, 11]
#   ]
# res = distribute(a)
# pprint(res)

In [ ]:
def build_column_names(col_num):
    """Return a list of column names."""
    columns = ['f1(x)', 'f2(x)', 'F12(A)', 'F12_dist']
    n = int((col_num - 4) / 3)
    count = 3
    for i in range(n):
        s = f'f{count}(x)'
        columns.append(s)
        r = ''.join([str(i) for i in range(1, count+1)])
        s = f'F{r}(A)'
        columns.append(s)
        s = f'F{r}_dist'
        columns.append(s)
        count += 1
    return columns

In [ ]:
# Часть 1 и 2
a = [[5, 7.5, 9, 10],
  [4, 5.5, 8, 10],
  [6, 8, 10, 12],
  [4.5, 7, 9, 11]
  ]
def main(a):
    res = distribute(a)
    dist_table = np.array(res, dtype='object')
    columns = build_column_names(dist_table.shape[0])
    best_dist = pd.DataFrame(dist_table.T, columns=columns)
    return best_dist

best_dist = main(a)
best_dist

,f1(x),f2(x),F12(A),F12_dist,f3(x),F123(A),F123_dist,f4(x),F1234(A),F1234_dist
0,0,0,0,"[0, 0]",0,0,"[0, 0, 0]",0,0,"[0, 0, 0, 0]"
1,5,4,5,"[1, 0]",6,6,"[0, 0, 1]",4.5,6,"[0, 0, 1, 0]"
2,7.5,5.5,9,"[1, 1]",8,11,"[1, 0, 1]",7,11,"[1, 0, 1, 0]"
3,9,8,11.5,"[2, 1]",10,15,"[1, 1, 1]",9,15.5,"[1, 0, 1, 1]"
4,10,10,13,"[3, 1]",12,17.5,"[2, 1, 1]",11,19.5,"[1, 1, 1, 1]"


In [ ]:
# часть 3
a = [
  # [5, 7.5, 9, 10],  # убираем 1-ый агрегат
  [4, 5.5, 8, 10],
  [6, 8, 10, 12],
  [4.5, 7, 9, 11]
  ]
best_dist = main(a)
best_dist

,f1(x),f2(x),F12(A),F12_dist,f3(x),F123(A),F123_dist
0,0,0,0,"[0, 0]",0,0,"[0, 0, 0]"
1,4,6,6,"[0, 1]",4.5,6,"[0, 1, 0]"
2,5.5,8,10,"[1, 1]",7,10.5,"[0, 1, 1]"
3,8,10,12,"[1, 2]",9,14.5,"[1, 1, 1]"
4,10,12,14,"[3, 1]",11,17,"[1, 1, 2]"


In [ ]:
# пример данных из учебника
b = [[0.28, 0.45, 0.65, 0.78, 0.9, 1.02, 1.13, 1.23, 1.32],
     [0.25, 0.41, 0.55, 0.65, 0.75, 0.8, 0.85, 0.88, 0.9],
     [0.15, 0.25, 0.4, 0.5, 0.62, 0.73, 0.82, 0.9, 0.96]
     ]

best_dist = main(b)
best_dist

,f1(x),f2(x),F12(A),F12_dist,f3(x),F123(A),F123_dist
0,0,0,0,"[0, 0]",0,0,"[0, 0, 0]"
1,0.28,0.25,0.28,"[1, 0]",0.15,0.28,"[1, 0, 0]"
2,0.45,0.41,0.53,"[1, 1]",0.25,0.53,"[1, 1, 0]"
3,0.65,0.55,0.7,"[2, 1]",0.4,0.7,"[2, 1, 0]"
4,0.78,0.65,0.9,"[3, 1]",0.5,0.9,"[3, 1, 0]"
5,0.9,0.75,1.06,"[3, 2]",0.62,1.06,"[3, 2, 0]"
6,1.02,0.8,1.2,"[3, 3]",0.73,1.21,"[3, 2, 1]"
7,1.13,0.85,1.33,"[4, 3]",0.82,1.35,"[3, 3, 1]"
8,1.23,0.88,1.45,"[5, 3]",0.9,1.48,"[4, 3, 1]"
9,1.32,0.9,1.57,"[6, 3]",0.96,1.6,"[5, 3, 1]"
